In [2]:
# source : https://towardsdatascience.com/build-your-own-transformer-from-scratch-using-pytorch-84c850470dcb

In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [4]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [5]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_model, num_heads):
        super(MultiHeadAttention, self).__init__()
        assert d_model % num_heads == 0, "d_model must be divisible by num_heads"

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

    def scaled_dot_product_attention(self, Q, K, V, mask=None):
        attn_scores = torch.matmul(Q, K.transpose(-2, -1)) / math.sqrt(self.d_k)
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)
        attn_probs = torch.softmax(attn_scores, dim=-1)
        output = torch.matmul(attn_probs, V)
        return output

    def split_heads(self, x):
        batch_size, seq_length, d_model = x.size()
        return x.view(batch_size, seq_length, self.num_heads, self.d_k).transpose(1, 2)

    def combine_heads(self, x):
        batch_size, _, seq_length, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_length, self.d_model)

    def forward(self, Q, K, V, mask=None):
        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        attn_output = self.scaled_dot_product_attention(Q, K, V, mask)
        output = self.W_o(self.combine_heads(attn_output))
        return output

In [6]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, d_model, d_ff):
        super(PositionWiseFeedForward, self).__init__()
        self.fc1 = nn.Linear(d_model, d_ff)
        self.fc2 = nn.Linear(d_ff, d_model)
        self.relu = nn.ReLU()

    def forward(self, x):
        return self.fc2(self.relu(self.fc1(x)))

In [7]:
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_seq_length):
        super(PositionalEncoding, self).__init__()

        pe = torch.zeros(max_seq_length, d_model).to(device)
        position = torch.arange(0, max_seq_length, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * -(math.log(10000.0) / d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        self.register_buffer('pe', pe.unsqueeze(0))

    def forward(self, x):
        return x + self.pe[:, :x.size(1)]

In [8]:
class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x

In [9]:
class DecoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(DecoderLayer, self).__init__()
        self.self_attn = MultiHeadAttention(d_model, num_heads)
        self.cross_attn = MultiHeadAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, enc_output, src_mask, tgt_mask):
        attn_output = self.self_attn(x, x, x, tgt_mask)
        x = self.norm1(x + self.dropout(attn_output))
        attn_output = self.cross_attn(x, enc_output, enc_output, src_mask)
        x = self.norm2(x + self.dropout(attn_output))
        ff_output = self.feed_forward(x)
        x = self.norm3(x + self.dropout(ff_output))
        return x

In [10]:
class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()
        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionalEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList(
            [EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList(
            [DecoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def generate_mask(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool().to(device)
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask

    def forward(self, src, tgt):
        src_mask, tgt_mask = self.generate_mask(src, tgt)
        src_embedded = self.dropout(self.positional_encoding(self.encoder_embedding(src)))
        tgt_embedded = self.dropout(self.positional_encoding(self.decoder_embedding(tgt)))

        enc_output = src_embedded
        for enc_layer in self.encoder_layers:
            enc_output = enc_layer(enc_output, src_mask)

        dec_output = tgt_embedded
        for dec_layer in self.decoder_layers:
            dec_output = dec_layer(dec_output, enc_output, src_mask, tgt_mask)

        output = self.fc(dec_output)
        return output

# [!] pas de decalage entre decode input et output?

# LZ77 Algorithm

In [11]:
# https://github.com/manassra/LZ77-Compressor

import math
from bitarray import bitarray


class LZ77Compressor:
    """
    A simplified implementation of the LZ77 Compression Algorithm
    """
    MAX_WINDOW_SIZE = 400

    def __init__(self, window_size=20):
        self.window_size = min(window_size, self.MAX_WINDOW_SIZE)
        self.lookahead_buffer_size = 15  # length of match is at most 4 bits

    def compress(self, input_file_path, output_file_path=None, verbose=False):
        """
        Given the path of an input file, its content is compressed by applying a simple
        LZ77 compression algorithm.

        The compressed format is:
        0 bit followed by 8 bits (1 byte character) when there are no previous matches
            within window
        1 bit followed by 12 bits pointer (distance to the start of the match from the
            current position) and 4 bits (length of the match)

        If a path to the output file is provided, the compressed data is written into
        a binary file. Otherwise, it is returned as a bitarray

        if verbose is enabled, the compression description is printed to standard output
        """
        data = None
        i = 0
        output_buffer = bitarray(endian='big')

        # read the input file
        try:
            with open(input_file_path, 'rb') as input_file:
                data = input_file.read()
        except IOError:
            print('Could not open input file ...')
            raise

        while i < len(data):
            # print(i)

            match = self.findLongestMatch(data, i)

            if match:
                # Add 1 bit flag, followed by 12 bit for distance, and 4 bit for the length
                # of the match
                (bestMatchDistance, bestMatchLength) = match

                output_buffer.append(True)
                output_buffer.frombytes(bytes([bestMatchDistance >> 4]))
                output_buffer.frombytes(bytes([((bestMatchDistance & 0xf) << 4) | bestMatchLength]))

                if verbose:
                    print("<1, %i, %i>" % (bestMatchDistance, bestMatchLength), end='')

                i += bestMatchLength

            else:
                # No useful match was found. Add 0 bit flag, followed by 8 bit for the character
                output_buffer.append(False)
                output_buffer.frombytes(bytes([data[i]]))

                if verbose:
                    print("<0, %s>" % data[i], end='')

                i += 1

        # fill the buffer with zeros if the number of bits is not a multiple of 8
        output_buffer.fill()

        # write the compressed data into a binary file if a path is provided
        if output_file_path:
            try:
                with open(output_file_path, 'wb') as output_file:
                    output_file.write(output_buffer.tobytes())
                    print("File was compressed successfully and saved to output path ...")
                    return None
            except IOError:
                print('Could not write to output file path. Please check if the path is correct ...')
                raise

        # an output file path was not provided, return the compressed data
        return output_buffer

    def findLongestMatch(self, data, current_position):
        """
        Finds the longest match to a substring starting at the current_position
        in the lookahead buffer from the history window
        """
        end_of_buffer = min(current_position + self.lookahead_buffer_size, len(data) + 1)

        best_match_distance = -1
        best_match_length = -1

        # Optimization: Only consider substrings of length 2 and greater, and just
        # output any substring of length 1 (8 bits uncompressed is better than 13 bits
        # for the flag, distance, and length)
        for j in range(current_position + 2, end_of_buffer):

            start_index = max(0, current_position - self.window_size)
            substring = data[current_position:j]

            for i in range(start_index, current_position):

                repetitions = len(substring) // (current_position - i)

                last = len(substring) % (current_position - i)

                matched_string = data[i:current_position] * repetitions + data[i:i + last]

                if matched_string == substring and len(substring) > best_match_length:
                    best_match_distance = current_position - i
                    best_match_length = len(substring)

        if best_match_distance > 0 and best_match_length > 0:
            return (best_match_distance, best_match_length)
        return None

    def compress_in_place(self, data, verbose=False):
        """
        Given the path of an input file, its content is compressed by applying a simple
        LZ77 compression algorithm.

        The compressed format is:
        0 bit followed by 8 bits (1 byte character) when there are no previous matches
            within window
        1 bit followed by 12 bits pointer (distance to the start of the match from the
            current position) and 4 bits (length of the match)

        If a path to the output file is provided, the compressed data is written into
        a binary file. Otherwise, it is returned as a bitarray

        if verbose is enabled, the compression description is printed to standard output
        """
        i = 0
        output_buffer = bitarray(endian='big')

        while i < len(data):

            # print(i)

            match = self.findLongestMatch(data, i)

            if match:
                # Add 1 bit flag, followed by 12 bit for distance, and 4 bit for the length
                # of the match
                (bestMatchDistance, bestMatchLength) = match

                output_buffer.append(True)
                output_buffer.frombytes(bytes([bestMatchDistance >> 4]))
                output_buffer.frombytes(bytes([((bestMatchDistance & 0xf) << 4) | bestMatchLength]))

                if verbose:
                    print("<1, %i, %i>" % (bestMatchDistance, bestMatchLength), end='')

                i += bestMatchLength

            else:
                # No useful match was found. Add 0 bit flag, followed by 8 bit for the character
                output_buffer.append(False)
                output_buffer.frombytes(bytes([data[i]]))

                if verbose:
                    print("<0, %s>" % data[i], end='')

                i += 1

        # fill the buffer with zeros if the number of bits is not a multiple of 8
        output_buffer.fill()

        # an output file path was not provided, return the compressed data
        return output_buffer

# functions for treat data

In [12]:
def bytes2torch(bytes):
    if not type(bytes) is list:
        data_integers = list(bytes)

    data_tensor = torch.tensor(data_integers, dtype=torch.uint8)

    return data_tensor


def tokenizer(byte_tensor, length, device):
    """"
    :param byte_tensor: torch tensor uint8
    :param length: length of the output, complete with padding if shorter
    :return: torch tensor int16
    """
    new_tensor = (byte_tensor[:length] + 1)
    zero_tensor = torch.zeros(length, dtype=torch.int64).to(device)
    zero_tensor[:new_tensor.size(0)] = new_tensor

    return zero_tensor


def tokenizer_for_transformer(byte_tensor, length, device, start_token=None, end_token=None):
    new_tensor = (byte_tensor[:length] + 1)

    if start_token and end_token:
        new_tensor = torch.cat([torch.tensor([start_token]), new_tensor, torch.tensor([end_token])])
    elif start_token:
        new_tensor = torch.cat([torch.tensor([start_token]), new_tensor])
    elif end_token:
        new_tensor = torch.cat([new_tensor, torch.tensor([end_token])])

    zero_tensor = torch.zeros(length, dtype=torch.int64).to(device)
    zero_tensor[:new_tensor.size(0)] = new_tensor

    return zero_tensor

## DataSet with files

In [13]:
import os
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from generate_data import NB_MAX_TOKEN

START_TOKEN_TRANSFORMER = NB_MAX_TOKEN
END_TOKEN_TRANSFORMER = NB_MAX_TOKEN + 1


# [!] arg changed
class LZ77Dataset(Dataset):
    def __init__(self, length, device, features_folder="./data_LZ77/text", targets_folder="./data_LZ77/compress",
                 for_transformer=False, start_token=None, end_token=None):
        self.features_folder = features_folder
        self.targets_folder = targets_folder
        self.for_transformer = for_transformer
        self.length = length
        self.device = device
        self.start_token = start_token
        self.end_token = end_token

        # Listes des noms de fichiers dans les dossiers
        self.feature_files = os.listdir(features_folder)
        self.target_files = os.listdir(targets_folder)

    def __len__(self):
        return min(len(self.feature_files), len(self.target_files))

    def __getitem__(self, idx):
        feature_path = os.path.join(self.features_folder, self.feature_files[idx])
        target_path = os.path.join(self.targets_folder, self.target_files[idx])

        with open(feature_path, 'rb') as feature_file:
            feature_data = feature_file.read()
        with open(target_path, 'rb') as target_file:
            target_data = target_file.read()

        if self.for_transformer:
            encoder_input = tokenizer_for_transformer(bytes2torch(feature_data), self.length, self.device,
                                                      start_token=START_TOKEN_TRANSFORMER,
                                                      end_token=END_TOKEN_TRANSFORMER)
            decoder_input = tokenizer_for_transformer(bytes2torch(target_data), self.length, self.device,
                                                      start_token=START_TOKEN_TRANSFORMER, end_token=None)
            feature_data = (encoder_input, decoder_input)
            target_data = tokenizer_for_transformer(bytes2torch(target_data), self.length, self.device,
                                                    start_token=None, end_token=END_TOKEN_TRANSFORMER)

        else:
            feature_data = tokenizer(bytes2torch(feature_data), self.length, self.device)
            target_data = tokenizer(bytes2torch(target_data), self.length, self.device)

        return feature_data, target_data

# function to create Data

In [14]:
import random
import string


def generate_random_ascii_bytes(length, repetition_probability=0.65):
    ascii_characters = string.ascii_letters + string.digits + string.punctuation
    dim = len(ascii_characters)
    random_ascii_text = ''

    for _ in range(length):
        if random.random() < repetition_probability and random_ascii_text:
            # Répétition d'un caractère existant
            random_ascii_text += random.choice(random_ascii_text)
        else:
            # Choix aléatoire d'un nouveau caractère
            random_ascii_text += random.choice(ascii_characters)

    # Convertir la chaîne de caractères en bytes en utilisant encode
    random_ascii_bytes = random_ascii_text.encode('utf-8')
    return random_ascii_bytes, dim

## Dataset Realtime

In [15]:
def create_sample_in_place(compressor, length, repetition_probability):
    text, dim = generate_random_ascii_bytes(length, repetition_probability)
    compress = compressor.compress_in_place(text)

    return text, compress


def bits_to_bytes(bits):
    return bits.tobytes()


class LZ77Dataset_current_creation(Dataset):
    def __init__(self, length_sentence, device, repetition_probability, size_data_set, lenght_max, for_transformer):
        self.compressor = LZ77Compressor()
        self.length_sentence = length_sentence
        self.device = device
        self.repetition_probability = repetition_probability
        self.size_data_set = size_data_set
        self.lenght_max = lenght_max
        self.for_transformer = for_transformer

    def __len__(self):
        return self.size_data_set

    def __getitem__(self, idx):
        text, compress = create_sample_in_place(self.compressor, self.length_sentence, self.repetition_probability)

        if self.for_transformer:
            encoder_input = tokenizer_for_transformer(bytes2torch(text), self.lenght_max, self.device,
                                                      start_token=START_TOKEN_TRANSFORMER,
                                                      end_token=END_TOKEN_TRANSFORMER)
            decoder_input = tokenizer_for_transformer(bytes2torch(bits_to_bytes(compress)), self.lenght_max,
                                                      self.device,
                                                      start_token=START_TOKEN_TRANSFORMER, end_token=None)
            feature_data = (encoder_input, decoder_input)
            target_data = tokenizer_for_transformer(bytes2torch(bits_to_bytes(compress)), self.lenght_max, self.device,
                                                    start_token=None, end_token=END_TOKEN_TRANSFORMER)

        else:
            feature_data = tokenizer(bytes2torch(text), self.lenght_max, self.device)
            target_data = tokenizer(bytes2torch(bits_to_bytes(compress)), self.lenght_max, self.device)

        # feature_data = tokenizer(bytes2torch(text), self.lenght_max, self.device)
        # target_data = tokenizer(bytes2torch(bits_to_bytes(compress)), self.lenght_max, self.device)

        return feature_data, target_data

# set train and test loader

In [17]:
# [!] padding avec zero a bien faire attention que identique
# [!] verifier que les to(device) sont dans les init ou a la creation
# [!] augmenter taille des batches

from torch.utils.data import random_split
from generate_data import SENTENCE_LENGTH

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

SENTENCE_LENGTH = 10  # [!] only if on realtime creation
NB_SPECIAL_TOKEN = 2
NB_MAX_TOKEN_TRANSFORMER = NB_MAX_TOKEN + NB_SPECIAL_TOKEN
MAX_LENGTH = SENTENCE_LENGTH + 5 + NB_SPECIAL_TOKEN
length_sentence = SENTENCE_LENGTH
repetition_probability = 0.8
size_data_set = 256
batch_size = 256
test_proportion = 0.8

lz77_dataset = LZ77Dataset_current_creation(length_sentence, device, repetition_probability, size_data_set, MAX_LENGTH,
                                            for_transformer=True)

test_size = int(test_proportion * len(lz77_dataset))

train_dataset, test_dataset = random_split(lz77_dataset, [len(lz77_dataset) - test_size, test_size])

lz77_train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
lz77_test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True)

for batch in lz77_train_loader:
    (txt_example, cmp_input_example), cmp_output_example = batch
    break

print("feature and target :", (txt_example[0], cmp_input_example[0]), cmp_output_example[0])

feature and target : (tensor([257,  43,  43,  43,  43,  43,  43,  92,  63,  63,  63, 258,   0,   0,
          0,   0,   0], device='cuda:0'), tensor([257,  22,  65,   6,  76, 100, 233,   1, 145,   0,   0,   0,   0,   0,
          0,   0,   0], device='cuda:0')) tensor([ 22,  65,   6,  76, 100, 233,   1, 145, 258,   0,   0,   0,   0,   0,
          0,   0,   0], device='cuda:0')


In [37]:
src_vocab_size = NB_MAX_TOKEN_TRANSFORMER
tgt_vocab_size = NB_MAX_TOKEN_TRANSFORMER
d_model = 512
num_heads = 8
num_layers = 6
d_ff = 2048
max_seq_length = MAX_LENGTH
dropout = 0.1

transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length,
                          dropout).to(device)

# Generate random sample data
# src_data = torch.randint(1, src_vocab_size, (64, max_seq_length)).to(device)  # (batch_size, seq_length)
# tgt_data = torch.randint(1, tgt_vocab_size, (64, max_seq_length)).to(device)  # (batch_size, seq_length)

# inference

In [75]:
def generate_sequences_batch(transformer, src_data, max_length=20):
    transformer.eval()  # Mettre le modèle en mode évaluation
    with torch.no_grad():
        batch_size = src_data.size(0)
        generated_sequences = [[START_TOKEN_TRANSFORMER] for _ in range(batch_size)]

        for _ in range(max_length):
            # Obtenez les prédictions pour la séquence actuelle
            generated_sequences_torch = [torch.tensor(seq).unsqueeze(0).to(dtype=torch.long).to(src_data.device) for seq
                                         in generated_sequences]
            output = transformer(src_data, torch.cat(generated_sequences_torch, dim=0))

            # Utiliser argmax pour obtenir l'indice du maximum du dernier predit
            next_tokens = torch.argmax(output[:, -1, :], dim=1)

            # Ajouter les prédictions aux séquences générées
            for i in range(batch_size):
                generated_sequences[i].append(next_tokens[i].item())
                

        return torch.tensor(generated_sequences).to(device)


# Exemple d'utilisation avec un batch de données source
src_data_batch = torch.tensor([[1, 2, 3, 4], [5, 6, 7, 8]]).to(dtype=torch.long).to(device)
generated_sequences_batch = generate_sequences_batch(transformer, src_data_batch, max_length=max_seq_length)
print("Generated Sequences Batch:", generated_sequences_batch)

Generated Sequences Batch: tensor([[257,   5,  30,  33,   5,  81,   5,  35,   5,   2,   5, 258, 258, 258,
         258,   1,   5, 258],
        [257,  61,  31,  16,   5,  81,   2,  34,   5,  17, 258, 258, 258, 258,
           1, 258,   1,   5]], device='cuda:0')


# metrics

In [48]:
def mean_error(batch_1, batch_2):
    errors = (batch_1 != batch_2).float()  # 1 si différent, 0 si identique
    error_rate = torch.mean(errors).item()  # Calculer la moyenne des erreurs
    return error_rate


batch_1 = torch.tensor([[1.0, 2.2, 3.1], [1.0, 2.0, 3.0]])
batch_2 = torch.tensor([[1.1, 2.4, 3.0], [1.0, 2.0, 3.0]])

print(mean_error(batch_1, batch_2))


def mean_error_transformer(transformer, txt_batch, cmp_output_batch, max_seq_length):
    generated_sequences_batch = generate_sequences_batch(transformer, txt_batch, max_length=max_seq_length)
    return mean_error(generated_sequences_batch, replace_padding_by_end_token(add_start_token(cmp_output_batch)))


max_mean_error = max_seq_length / (max_seq_length + 1) # [!] not really exact


0.5


In [69]:
def vector_to_one_hot(vector, vocabulary_size):
    """
    Convertit un vecteur d'indices en une matrice one-hot.

    Args:
        vector (torch.Tensor): Vecteur d'indices.
        vocabulary_size (int): Taille du vocabulaire.

    Returns:
        torch.Tensor: Matrice one-hot.
    """
    one_hot_matrix = torch.zeros(len(vector), vocabulary_size).to(device).float()
    one_hot_matrix.scatter_(1, vector.view(-1, 1), 1)
    return one_hot_matrix

# Exemple d'utilisation
vector = torch.tensor([2, 0, 1, 2, 1]).to(device)
vocabulary_size = 3  # Exemple, ajuste-le en fonction de la taille de ton vocabulaire
one_hot_matrix = vector_to_one_hot(vector, vocabulary_size)

print("Vecteur d'indices:")

print(vector)

print("\nMatrice one-hot:")
print(one_hot_matrix)


Vecteur d'indices:
tensor([2, 0, 1, 2, 1], device='cuda:0')

Matrice one-hot:
tensor([[0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [0., 0., 1.],
        [0., 1., 0.]], device='cuda:0')


In [66]:
def replace_padding_by_end_token(batch):
    return torch.where(batch == 0, END_TOKEN_TRANSFORMER, batch)

def add_start_token(batch):
    return torch.cat([torch.full((batch.size(0), 1), fill_value=START_TOKEN_TRANSFORMER).to(device), batch], dim=1)

In [76]:
generated_sequences_batch = generate_sequences_batch(transformer, txt_example[0:1, :], max_length=max_seq_length)

print("Generated Sequences Batch:", generated_sequences_batch)
print("target sentence: ", replace_padding_by_end_token(add_start_token(cmp_output_example[0:1, :])))
print(mean_error_transformer(transformer, txt_example[:, :], cmp_output_example[:, :], max_seq_length))

Generated Sequences Batch: tensor([[257,  22,  65,   6,  12, 113,   3,  34, 249, 258, 258, 258, 258, 258,
         258, 258, 258,   1]], device='cuda:0')
target sentence:  tensor([[257,  22,  65,   6,  76, 100, 233,   1, 145, 258, 258, 258, 258, 258,
         258, 258, 258, 258]], device='cuda:0')
0.3568376302719116


In [72]:
from tqdm import tqdm

# [!] on a :-1 en input et 1: en output mais pas de start et end token => necessaire?

criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()
size_batch_for_train = 10

first = True

for epoch in range(2000):
    for (txt, cmp_input), cmp_output in tqdm(lz77_train_loader):
        if first:
            print(
                f"mean error : {mean_error_transformer(transformer, txt_example[:10, :], cmp_output_example[:10, :], max_seq_length):.4f}/{max_mean_error:.4f}")
            first = False
        optimizer.zero_grad()
        output = transformer(txt, cmp_input)
        prediction = output.contiguous().view(-1, tgt_vocab_size)
        target = cmp_output.contiguous().view(-1)
        loss = criterion(prediction, target)
        loss.backward()
        optimizer.step()
        torch.save(transformer.state_dict(), "save_model_transformer_2/v_1")
    print(f"Epoch: {epoch + 1}, Loss: {loss.item()}")
    print(
        f"mean error : {mean_error_transformer(transformer, txt_example[:10, :], cmp_output_example[:10, :], max_seq_length):.4f}/{max_mean_error:.4f}")

  0%|          | 0/1 [00:00<?, ?it/s]

mean error : 0.3833/0.9444


100%|██████████| 1/1 [00:01<00:00,  1.73s/it]


Epoch: 1, Loss: 2.3554611206054688
mean error : 0.3278/0.9444


100%|██████████| 1/1 [00:00<00:00,  2.17it/s]


...

100%|██████████| 1/1 [00:00<00:00,  1.30it/s]


Epoch: 260, Loss: 1.5682287216186523
mean error : 0.2278/0.9444


100%|██████████| 1/1 [00:00<00:00,  1.46it/s]


Epoch: 261, Loss: 1.5105743408203125


KeyboardInterrupt: 